# Google Trends Data using [pytrends](https://github.com/GeneralMills/pytrends)

---

In [1]:
# === Connect to Google === #
from pytrends.request import TrendReq

# Mountain Time offset is -7:00 = 420 minutes
pytrends = TrendReq(hl="en-US", tz=420)

In [33]:
# === Other imports === #
import pandas as pd
import numpy as np
import os
import time

In [100]:
# === Build payload === #

# Base word to include in query
recycle = "recycle"

# List of materials to query
keywords = [
    'crayons',
]

# Concatenate each keyword with the base word
concat_kw = [" ".join([recycle, kw]) for kw in keywords]

print(concat_kw)

['recycle crayons']


In [101]:
# === Build payload, Pt. II === #
pytrends.build_payload(
    concat_kw,
)

In [102]:
# === Return dataframe of interest over time === #
df1 = pytrends.interest_over_time()
df1.head()

,recycle crayons,isPartial
date,,
2015-01-25,0,False
2015-02-01,40,False
2015-02-08,0,False
2015-02-15,41,False
2015-02-22,40,False


In [8]:
df1.tail()

,recycle crayons,isPartial
date,,
2019-12-22,24,False
2019-12-29,22,False
2020-01-05,27,False
2020-01-12,39,False
2020-01-19,0,True


In [103]:
# === Export dataframe with date index only === #
df_dates_blank = df1.drop(columns=["recycle crayons", "isPartial"])
df_dates_blank.head()

""
date
2015-01-25
2015-02-01
2015-02-08
2015-02-15
2015-02-22


In [104]:
df_dates_blank.to_csv("blank_date_index.csv")

---

### Automate

The goal here is to...

- Iterate through the full list of keywords
- Send off 5 at a time to Google Trends
- Concatenate the resulting data into a single dataframe

In [22]:
# === Import items into list === #

# Open text file and read, splitting on the newline character
with open("earth911_items_list.txt", "r") as f:
    recycle_list = f.read().splitlines()

In [23]:
len(recycle_list)

332

In [75]:
recycle_list[45:60]

['Acids',
 'Adhesives',
 'Adult Toys',
 'Aerosol Cans - Full',
 'Air Conditioners',
 'Alkaline Batteries',
 'Aluminum Beverage Cans',
 'Aluminum Foil',
 'Aluminum Food Cans',
 'Aluminum Pie Plates',
 'Aluminum Trays',
 'Ammunition',
 'Antifreeze',
 'Arts and Crafts Supplies',
 'Asbestos']

In [32]:
# === Test out slicing list into 5 at a time === #

s_prev = 0

for s in range(0, 336, 5):
    # Slice from prev value of s to current
    recycle_list[s_prev:s]
    # Set current value of s to use in next iteration
    s_prev = s

In [77]:
def build_trend_df(base_query: str, kw_list: list, df) -> None:
    """
    Google Trends for 5 keywords at a time and add to dataframe.
    
    Parameters
    ----------
    base_query : string
        Query onto which each keyword is appended.
    kw_list : list
        List of keywords.
    df : pd.DataFrame
        Pre-existing dataframe onto which trends data will be added.
    
    Returns
    -------
    None
        Trends data is added to pre-existing dataframe, not returned.
    """

    s_prev = 45  # Used to keep track of previous loop value
    # Build lists of 5 items at a time
    for s in range(50, 336, 5):
        five_items_list = kw_list[s_prev:s]  # Slice from prev value of s to current

        # Concatenate each keyword with the base word - returns new list
        concat_queries = [" ".join([base_query, kw]) for kw in five_items_list]
#         print(concat_queries)

        # Build the payload and run the query for current 5 items
        pytrends.build_payload(concat_queries)
        df_return = pytrends.interest_over_time()

        # Add current results to OG dataframe
        if df_return.empty:
            print(f"No results for {s_prev}-{s}.")
        else:
            for item in concat_queries:
                if item in df_return.columns:
                    df[item] = df_return[item]

        # Space out the requests and give visual progress status
        print(f"Completed items {s_prev}-{s}.")
        time.sleep(2)
        s_prev = s  # Set current value of s to use in next iteration

In [45]:
# === Create OG Dataframe === #
df_og = df1.drop(columns=["recycle crayons", "isPartial"])
df_og

""
date
2015-01-25
2015-02-01
2015-02-08
2015-02-15
2015-02-22
...
2019-12-22
2019-12-29
2020-01-05


In [78]:
# === Run the function! === #
base_q1 = "recycle"
base_q2 = "how to recycle"

build_trend_df(base_q1, recycle_list, df_og)

Completed items 45-50
Completed items 50-55
Completed items 55-60
Completed items 60-65
No results.
Completed items 65-70
Completed items 70-75
Completed items 75-80
Completed items 80-85
Completed items 85-90
Completed items 90-95
Completed items 95-100
Completed items 100-105
Completed items 105-110
Completed items 110-115
Completed items 115-120
Completed items 120-125
Completed items 125-130
Completed items 130-135
Completed items 135-140
Completed items 140-145
Completed items 145-150
No results.
Completed items 150-155
Completed items 155-160
Completed items 160-165
No results.
Completed items 165-170
No results.
Completed items 170-175
No results.
Completed items 175-180
No results.
Completed items 180-185
Completed items 185-190
Completed items 190-195
Completed items 195-200
Completed items 200-205
No results.
Completed items 205-210
Completed items 210-215
Completed items 215-220
Completed items 220-225
Completed items 225-230
Completed items 230-235
Completed items 235-240
N

In [84]:
# === Check out the results === #
df_og.describe()
# df_og.head()

,recycle Acids,recycle Adhesives,recycle Adult Toys,recycle Aerosol Cans - Full,recycle Air Conditioners,recycle Alkaline Batteries,recycle Aluminum Beverage Cans,recycle Aluminum Foil,recycle Aluminum Food Cans,recycle Aluminum Pie Plates,...,recycle Weeds,recycle Wet-strength Paperboard,recycle Windows,recycle Wood,recycle Wood Chips,recycle Wood Furnishings,recycle Wood Stains,recycle X-rays,recycle Yard Waste,recycle Yoga Mats
count,261.0,261.0,261.0,261.000000,261.0,261.000000,261.0,261.000000,261.0,261.0,...,261.0,261.0,261.000000,261.000000,261.0,261.0,261.0,261.0,261.000000,261.0
mean,0.0,0.0,0.0,25.199234,0.0,44.877395,0.0,36.015326,0.0,0.0,...,0.0,0.0,52.302682,11.356322,0.0,0.0,0.0,0.0,31.390805,0.0
std,0.0,0.0,0.0,17.222709,0.0,16.707676,0.0,19.192540,0.0,0.0,...,0.0,0.0,15.322318,4.304676,0.0,0.0,0.0,0.0,14.039524,0.0
min,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,19.000000,2.000000,0.0,0.0,0.0,0.0,0.000000,0.0
25%,0.0,0.0,0.0,21.000000,0.0,33.000000,0.0,24.000000,0.0,0.0,...,0.0,0.0,41.000000,8.000000,0.0,0.0,0.0,0.0,22.000000,0.0
50%,0.0,0.0,0.0,24.000000,0.0,43.000000,0.0,33.000000,0.0,0.0,...,0.0,0.0,51.000000,11.000000,0.0,0.0,0.0,0.0,30.000000,0.0
75%,0.0,0.0,0.0,33.000000,0.0,54.000000,0.0,48.000000,0.0,0.0,...,0.0,0.0,60.000000,14.000000,0.0,0.0,0.0,0.0,39.000000,0.0
max,0.0,0.0,0.0,100.000000,0.0,100.000000,0.0,92.000000,0.0,0.0,...,0.0,0.0,100.000000,25.000000,0.0,0.0,0.0,0.0,100.000000,0.0


In [97]:
# === Export to csv for posterity === #
df_og.to_csv("earth911_items_pytrends_all.csv")

In [85]:
# === New df to filter out 0-value columns === #
df2 = df_og.copy()

In [91]:
# === Get list of 0-value columns === #

zero_value_cols = []

for col in df2.columns:
    if df2[col].mean() == 0.0:
        zero_value_cols.append(col)

In [92]:
# === Drop 0-value columns === #
df3 = df2.drop(columns=zero_value_cols)
df3.shape

(261, 78)

In [95]:
# === Result is 78 columns - Check em out === #
pd.options.display.max_rows = 100
df3.describe().T

,count,mean,std,min,25%,50%,75%,max
recycle Aerosol Cans - Full,261.0,25.199234,17.222709,0.0,21.0,24.0,33.0,100.0
recycle Alkaline Batteries,261.0,44.877395,16.707676,0.0,33.0,43.0,54.0,100.0
recycle Aluminum Foil,261.0,36.015326,19.192540,0.0,24.0,33.0,48.0,92.0
recycle Antifreeze,261.0,28.513410,17.455130,0.0,17.0,26.0,38.0,100.0
recycle Asphalt,261.0,23.409962,14.915549,0.0,19.0,21.0,31.0,80.0
recycle Auto Parts,261.0,45.183908,16.390097,0.0,32.0,44.0,57.0,100.0
recycle Brick,261.0,32.252874,20.910184,0.0,27.0,31.0,42.0,100.0
recycle Brush,261.0,14.984674,14.316738,0.0,0.0,16.0,22.0,100.0
recycle CD Cases,261.0,17.494253,13.490059,0.0,0.0,17.0,23.0,70.0
recycle CDs,261.0,33.045977,15.236526,0.0,22.0,31.0,39.0,100.0


In [96]:
# === Export pruned df to csv (for posterity!) === #
df3.to_csv("earth911_items_pytrends.csv")